In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#Load the dataset
#file_path = os.path.join(os.getcwd(), "Data", "raw", "student_depression_dataset.csv")
#df = pd.read_csv(file_path)

df=pd.read_csv("./Data/raw/student_depression_dataset.csv")

In [14]:
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,'5-6 hours',Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,'5-6 hours',Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,'Less than 5 hours',Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,'7-8 hours',Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,'5-6 hours',Moderate,M.Tech,Yes,1.0,1.0,No,0


Check the overall details about the dataset

In [9]:
# Print basic info about the dataset
print(f"   Information of the raw dataset \n -------------------------------------------\n")
df.info()

   Information of the raw dataset 
 -------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  fl

Check whether there are missing values in the dataset

In [10]:
#Check missing values
df.isnull().sum()

id                                       0
Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64

Check whether there are duplicates in the dataset

In [11]:
#Check for duplicate rows
df.duplicated().sum()

np.int64(0)

Drop Irrelevant Columns

In [17]:
df.drop(columns=['id'],inplace=True)

Identify Feature Types

In [18]:
#Target Variable
target='Depression'

#Features(excluding target)
features=df.drop(columns=[target])

#Categorical Features
categorical_cols=features.select_dtypes(include='object').columns.tolist()

#Numerical Features
numerical_cols=features.select_dtypes(exclude='object').columns.tolist()

print("Categorical:",categorical_cols)
print("Numerical:",numerical_cols)

Categorical: ['Gender', 'City', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Financial Stress', 'Family History of Mental Illness']
Numerical: ['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours']


Encode Categorical Variables
SVM only works with numerical data. So need to encode categorical data

01.Label Encode Binary Categorical Columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


#Apply label encoding to binary categorical columns
binary_cols=['Gender','Academic Pressure','Work Pressure',
               'Study Satisfaction','Job Satisfaction',
               'Dietary Habits','Have you ever had suicidal thoughts ?',
               'Financial Stress','Family History of Mental Illness']

for col in binary_cols:
    df[col] =le.fit_transform(df[col])

02.One-Hot encode nominal categorical Columns

In [ ]:
df=pd.get_dummies(df,columns=['City','Profession','Degree'],drop_first=True)
